In [1]:
##### EDS biasing using HTF #####
## Step 1: Build training graph. ##
## Here the collective variable (CV) is the average distance to center of mass##
## Step 2: Set up Hoomd-Sim of arbitrary particle 'A' ##

In [ ]:
import hoomd.htf as htf
import sys
import tensorflow as tf
import hoomd
import hoomd.md

In [1]:
#### Build training graph ####

def make_eds_graph(N, NN, directory,set_pt):
    #N= Number of atoms in the system, NN=Number of nearest neighbors, 
    #directory=directory to save the results, 
    #set_pt=set point in EDS method
    '''Currently only computes running mean'''

    graph =htf.graph_builder(NN,output_forces=True)
    #calculate center of mass
    com = tf.reduce_mean(graph.positions[:, :2], 0) 
    #calculate distance of each atom from center of mass
    rs = graph.safe_norm(tf.math.subtract(graph.positions[:, :2], com), axis=1) 
    #calculate the average distance from center of mass. This is the collective variable (CV)
    real_cv = tf.reduce_mean(rs) 
    #calculates the running mean of the CV
    cv_run=graph.running_mean(tensor=real_cv,name='cv_run') 
    #calculate the EDS alpha value every 300 steps. See htf/utils.py for more information
    eds_alpha = htf.eds_bias(real_cv, set_point=set_pt, period=300,learning_rate=5.0)
    eds_energy = eds_alpha * real_cv #computes EDS energy
    #compute EDS forces
    eds_forces = graph.compute_forces(eds_energy) 
    #write a scalar summary of CV for tensorboard. Make sure write_tensorboard=True in hoomd-sim code
    tf.summary.scalar('collective variable',cv_run) 
    printer1=tf.Print(cv_run,[cv_run],message='CV')
    printer2=tf.Print(eds_alpha,[eds_alpha],message='a')
    graph.save(model_directory=directory,out_nodes=[[printer1,100],[printer2,100]],force_tensor=eds_forces, virial=None)


In [ ]:
### Hoomd-Sim code ###

model_dir = '/home/your/working/directory/'

make_eds_graph(64, 30, model_dir, 4.0)

#write data to tensorboard in realtime
with hoomd.htf.tfcompute(model_dir,write_tensorboard=True) as tfcompute: 
    hoomd.context.initialize()
    N = 64
    NN = 30
    #cut off radius: must be less than the box size
    rcut = 6.0 
    #initialize the lattice
    system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sq(a=2.0),n=[8, 8]) 
    hoomd.md.update.enforce2d()
    nlist = hoomd.md.nlist.cell(check_period=1)
    #enable lj pair potential
    lj = hoomd.md.pair.lj(rcut, nlist) 
    #set lj coefficients
    lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0) 
    hoomd.md.integrate.mode_standard(dt=0.005)
    # set up NVT simulation
    hoomd.md.integrate.nvt(kT=1.0, tau=0.5,group=hoomd.group.all()) 
    hoomd.analyze.log(filename='eds.log',quantities=['momentum', 'temperature', 'time'],period=100, overwrite=True) #set up output log
    
    #equilibrate
    hoomd.run(3000)

    #simulation                  
    tfcompute.attach(nlist, r_cut=rcut,)
    hoomd.dump.gsd(filename='saveall_running.gsd',overwrite=True, period=100, group=hoomd.group.all(),dynamic=['attribute', 'momentum', 'topology']) #dump in gsd format
    hoomd.run(10000)

print('Training Done')


In [ ]:
##### Multidimensional EDS biasing using HTF #####
## Similar steps as above. Instead of one CV now we have 2 CVs ## 
## CVs are: 1. average distance between particles and the center of mass along x axis
#           2. average distance between particles and the center of mass along y axis

In [ ]:
#### Build training graph for multidimensional EDS####

def make_eds_graph(N, NN, directory,set_pt1,set_pt2):
    #N= Number of atoms in the system, NN=Number of nearest neighbors, 
    #directory=directory to save the results, 
    #set_pt=set point in EDS method
    '''Currently only computes running mean'''

    graph =htf.graph_builder(NN,output_forces=True)
    #calculate center of mass
    com = tf.reduce_mean(graph.positions[:, :2], 0)
    #calculate distance of each atom from center of mass along x axis
    r_x = tf.math.subtract(graph.positions[:, 0], tf.ones_like(graph.positions[:, 0])*com[0])
    #calculate distance of each atom from center of mass along y axis
    r_y = tf.math.subtract(graph.positions[:, 1], tf.ones_like(graph.positions[:, 1])*com[1])
    #calculate CV 1: average distance along x axis 
    real_cv1 = tf.reduce_mean(r_x)
    #calculate CV 2: average distance along y axis 
    real_cv2 = tf.reduce_mean(r_y)
    #calculates the running mean of the CVs
    cv_run1=graph.running_mean(tensor=real_cv1,name='cv_run1')
    cv_run2=graph.running_mean(tensor=real_cv2,name='cv_run2')

    #Calculate the EDS alpha value for each CV, every 300 steps. 
    #See htf/utils.py for more information
    eds_alpha1 = htf.eds_bias(real_cv1, set_point=set_pt1, period=300,learning_rate=5.0)
    eds_alpha2 = htf.eds_bias(real_cv2, set_point=set_pt2, period=300,learning_rate=5.0)
    eds_energy = (eds_alpha1 * real_cv1) + (eds_alpha2 * real_cv2) #computes EDS energy                     
    #compute EDS forces
    eds_forces = graph.compute_forces(eds_energy) 
    
    printer1=tf.Print(cv_run1,[cv_run1],message='CV1')
    printer2=tf.Print(cv_run2,[cv_run2],message='CV2')                     
    printer3=tf.Print(eds_alpha1,[eds_alpha1],message='a1')
    printer4=tf.Print(eds_alpha2,[eds_alpha2],message='a2')
                         
    graph.save(model_directory=directory,out_nodes=[[printer1,100],[printer2,100],[printer4,100],[printer4,100]],force_tensor=eds_forces, virial=None)


In [ ]:
### Hoomd-Sim code ###

model_dir = '/home/your/working/directory/'

make_eds_graph(64, 30, model_dir, 4.0, 4.0)


with hoomd.htf.tfcompute(model_dir,write_tensorboard=False) as tfcompute: 
    hoomd.context.initialize()
    N = 64
    NN = 30
    #cut off radius: must be less than the box size
    rcut = 6.0 
    #initialize the lattice
    system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sq(a=2.0),n=[8, 8]) 
    hoomd.md.update.enforce2d()
    nlist = hoomd.md.nlist.cell(check_period=1)
    #enable lj pair potential
    lj = hoomd.md.pair.lj(rcut, nlist) 
    #set lj coefficients
    lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0) 
    hoomd.md.integrate.mode_standard(dt=0.005)
    # set up NVT simulation
    hoomd.md.integrate.nvt(kT=1.0, tau=0.5,group=hoomd.group.all()) 
    hoomd.analyze.log(filename='eds.log',quantities=['momentum', 'temperature', 'time'],period=100, overwrite=True) #set up output log
    
    #equilibrate
    hoomd.run(3000)

    #simulation                  
    tfcompute.attach(nlist, r_cut=rcut,)
    hoomd.dump.gsd(filename='saveall_running.gsd',overwrite=True, period=100, group=hoomd.group.all(),dynamic=['attribute', 'momentum', 'topology']) #dump in gsd format
    hoomd.run(10000)

print('Training Done')